<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [54]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [55]:
# extend stopwords to include extremely common words specific to the works
STOPWORDS = set(STOPWORDS).union(set(['food', 'place', 'time', ]))

def tokenize(doc):
    '''
    tokenizer
    '''  
    return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [56]:
def create_df(df):
    '''
    tokenize and keep only key columns
    '''
    df2 = df.copy()
    df2['tokens'] = df2['text'].apply(tokenize)
    return df2

yelp = create_df(yelp)
yelp.head()    

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[came, lunch, togo, service, quick, staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[ve, vegas, dozens, times, stepped, foot, circ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[went, night, closed, street, party, best, act..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[stars, bad, price, lunch, seniors, pay, eatin..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

data = yelp['text']

# instantiate count
vect = CountVectorizer(stop_words='english', min_df = 0.05, max_df= 0.90)

# tokenize and build vocab
vect.fit(data)

# transform text
sparse_dtm = vect.transform(data)

dtm = pd.DataFrame(sparse_dtm.todense(), columns=vect.get_feature_names())

In [79]:
dtm.head()

,10,amazing,area,asked,away,awesome,bad,bar,best,better,...,vegas,wait,want,wanted,wasn,way,went,work,worth,years
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# Instantiate
from sklearn.neighbors import NearestNeighbors

# Fit on Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [81]:
sample_review = ["""We have never had such awful service. They first mixed up our order with another table's, then the food came out cold, there was a bug in it, the drinks never arrived. our reservation was not honoured. I will not be returning here again."""]


In [82]:
# Query for Sim of Random doc to Yelp
new = vect.transform(sample_review)

nn.kneighbors(new.todense())

(array([[1.73205081, 2.        , 2.        , 2.        , 2.        ]]),
 array([[1986, 1104, 2895, 7242,  829]]))

In [83]:
# inspect most relevant result
data[1986]

'While the food and drinks are okay the service is virtually nonexistent.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [84]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

In [85]:
# define pipeline components 

# note: vec can be any Tfidf or Count Vectorizer
vect = CountVectorizer(stop_words='english', min_df = 0.05, max_df= 0.90)
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [86]:
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   47.5s
[Parallel(n_jobs=4)]: Done 160 out of 160 | elapsed:  2.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=0.05,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
     ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [118]:
grid_search.predict(["'We have never had such awful service. They first mixed up our order with another table's, then the food came out cold, there was a bug in it, the drinks never arrived. our reservation was not honoured. I will not be returning here again.'"])

array([1])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [119]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

Learn the vocubalary of the yelp data:

In [120]:
id2word = corpora.Dictionary(yelp['tokens'])


Create a bag of words representation of the entire corpus

In [121]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]


Your LDA model should be ready for estimation: 

In [122]:
import warnings
warnings.filterwarnings('ignore')

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=3,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [123]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [124]:
# evaluate the model
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.002233 -0.001863       1        1  10.158951
5     -0.001674  0.002068       2        1  10.122995
7     -0.000370 -0.000532       3        1  10.117035
1     -0.003702  0.002014       4        1  10.102073
8     -0.001529 -0.003817       5        1  10.040361
9      0.001275  0.002977       6        1  10.009675
6      0.001772  0.000420       7        1   9.954050
0      0.000685 -0.000903       8        1   9.870989
3      0.002293  0.001622       9        1   9.859021
2      0.003484 -0.001986      10        1   9.764853, topic_info=     Category         Freq        Term        Total  loglift  logprob
133   Default  4554.000000       place  4554.000000  30.0000  30.0000
144   Default  4762.000000        food  4762.000000  29.0000  29.0000
40    Default  4458.000000        good  4458.000000  28.0000  28.0000
208   Default  4132.000000       great  4132.000000  27.0000  27.0000
34    Default  3301.000000        time  3301.000000  26.0000  26.0000
131   Default  1382.000000      people  1382.000000  25.0000  25.0000
358   Default  1395.000000  definitely  1395.000000  24.0000  24.0000
45    Default  3505.000000     service  3505.000000  23.0000  23.0000
394   Default   761.000000       pizza   761.000000  22.0000  22.0000
115   Default  1860.000000          ve  1860.000000  21.0000  21.0000
117   Default  1434.000000        went  1434.000000  20.0000  20.0000
371   Default   905.000000       fresh   905.000000  19.0000  19.0000
1277  Default  1344.000000     chicken  1344.000000  18.0000  18.0000
192   Default  1314.000000     amazing  1314.000000  17.0000  17.0000
156   Default  1023.000000      pretty  1023.000000  16.0000  16.0000
313   Default   995.000000        want   995.000000  15.0000  15.0000
180   Default  1194.000000        menu  1194.000000  14.0000  14.0000
111   Default   826.000000       times   826.000000  13.0000  13.0000
138   Default   804.000000         bit   804.000000  12.0000  12.0000
735   Default  1588.000000        love  1588.000000  11.0000  11.0000
207   Default  1107.000000       going  1107.000000  10.0000  10.0000
264   Default  1250.000000  experience  1250.000000   9.0000   9.0000
92    Default  1687.000000        nice  1687.000000   8.0000   8.0000
46    Default  1506.000000       staff  1506.000000   7.0000   7.0000
41    Default   740.000000       lunch   740.000000   6.0000   6.0000
1066  Default  1118.000000         new  1118.000000   5.0000   5.0000
246   Default  1055.000000       right  1055.000000   4.0000   4.0000
351   Default  1401.000000        come  1401.000000   3.0000   3.0000
122   Default  1750.000000        best  1750.000000   2.0000   2.0000
75    Default  1989.000000         got  1989.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
37    Topic10   162.115448        came  1441.536743   0.1412  -5.6679
117   Topic10   161.279724        went  1434.414795   0.1410  -5.6731
238   Topic10    73.994194        away   611.269653   0.2148  -6.4522
599   Topic10    57.823750      review   466.035889   0.2395  -6.6988
180   Topic10   133.609360        menu  1194.288086   0.1360  -5.8613
1266  Topic10   161.246017     ordered  1488.049927   0.1041  -5.6733
1093  Topic10    58.304893         use   470.815094   0.2376  -6.6905
18    Topic10   321.264130        like  3329.338867  -0.0119  -4.9840
341   Topic10    75.254181         big   630.264282   0.2011  -6.4354
40    Topic10   410.606506        good  4458.327148  -0.0585  -4.7386
387   Topic10   153.359299       order  1449.676147   0.0801  -5.7234
36    Topic10   104.673996        wait   942.226746   0.1290  -6.1054
208   Topic10   363.122925       great  4132.155273  -0.1054  -4.8615
46    Topic10   153.989090       staff  1506.313232   0.0458  -5.7193
421   Topic10    91.134743       asked   808.734009   0.1432  -6.2439
8     Topic10    72

In [125]:
# each topic has a different distribution of the same "stock" of words
lda.print_topics()

[(0,
  '0.011*"great" + 0.011*"place" + 0.011*"food" + 0.010*"good" + 0.007*"service" + 0.007*"like" + 0.006*"time" + 0.004*"ve" + 0.004*"got" + 0.003*"little"'),
 (1,
  '0.011*"place" + 0.009*"service" + 0.009*"good" + 0.008*"time" + 0.007*"like" + 0.006*"food" + 0.006*"great" + 0.004*"got" + 0.004*"nice" + 0.004*"people"'),
 (2,
  '0.013*"food" + 0.009*"good" + 0.009*"service" + 0.008*"great" + 0.007*"place" + 0.007*"like" + 0.005*"got" + 0.005*"time" + 0.005*"ve" + 0.003*"came"'),
 (3,
  '0.010*"great" + 0.010*"food" + 0.009*"good" + 0.007*"time" + 0.007*"place" + 0.007*"service" + 0.006*"like" + 0.004*"got" + 0.004*"best" + 0.004*"ve"'),
 (4,
  '0.013*"place" + 0.009*"time" + 0.009*"food" + 0.008*"good" + 0.008*"great" + 0.007*"service" + 0.007*"like" + 0.005*"love" + 0.004*"got" + 0.004*"best"'),
 (5,
  '0.012*"good" + 0.012*"place" + 0.009*"food" + 0.009*"great" + 0.008*"like" + 0.007*"time" + 0.006*"service" + 0.004*"best" + 0.004*"nice" + 0.004*"staff"'),
 (6,
  '0.010*"great" 

In [126]:
import re
# use regex to remove the non-text components
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [127]:
# top 10 words
topics = [' '.join(t[0:10]) for t in words]

In [128]:
# print out nicely 
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
great place food good service like time ve got little

------ Topic 1 ------
place service good time like food great got nice people

------ Topic 2 ------
food good service great place like got time ve came

------ Topic 3 ------
great food good time place service like got best ve

------ Topic 4 ------
place time food good great service like love got best

------ Topic 5 ------
good place food great like time service best nice staff

------ Topic 6 ------
great food good place like service time went ve love

------ Topic 7 ------
place food great good like service time got people staff

------ Topic 8 ------
food time place service great like good nice ve definitely

------ Topic 9 ------
good food service place great time like ve amazing got



## Analysis:

A lot of reviews use generic terms like 'food', 'great', 'service' to describe their experiences. 

I think this is indicative of how reviews are weighted towards negative experiences. A postive experience is less likely to be reported at all, and if it is, reviews use the generic terms listed above. A negative experience is more likely to be reported and elaborated on at length with specifics of what did not meet expectations. 

Hence, as an analysis of how frequent words appear, more positive or generic words appear more frequently than compared to the large variatation in words chosen for negative reviews. 




## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)